In [1]:
# Ignore this block -- it's for the documentation build
try:
    import os, sys
    sys.path.insert(1, os.path.abspath('/home/runner/work/psi4/psi4/code/objdir/stage//usr/local/psi4/lib/'))
except ImportError:
    pass

# This is the important part
import psi4

In [2]:
psi4.set_output_file('output.dat', False)


PosixPath('output.dat')

In [3]:
ls

Assignment.md  output.dat  output.log  psi4.ipynb


In [4]:
#! Sample HF/cc-pVDZ H2O Computation

psi4.set_memory('500 MB')

h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
""")

psi4.energy('scf/cc-pvdz')

-76.02663273509214

In [5]:
#! Sample UHF/6-31G** CH2 Computation

R = 1.075
A = 133.93

ch2 = psi4.geometry("""
0 3
C
H 1 {0}
H 1 {0} 2 {1}
""".format(R, A)
)

psi4.set_options({'reference': 'uhf'})
psi4.energy('scf/6-31g**')

-38.92533462893768

In [6]:
psi4.set_options({'reference': 'rhf'})
psi4.optimize('scf/cc-pvdz', molecule=h2o)

Optimizer: Optimization complete!


-76.02703272937688

In [7]:
scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=h2o, return_wfn=True)

In [8]:
# Example SAPT computation for ethene*ethyne (*i.e.*, ethylene*acetylene).
# Test case 16 from S22 Database

dimer = psi4.geometry("""
0 1
C   0.000000  -0.667578  -2.124659
C   0.000000   0.667578  -2.124659
H   0.923621  -1.232253  -2.126185
H  -0.923621  -1.232253  -2.126185
H  -0.923621   1.232253  -2.126185
H   0.923621   1.232253  -2.126185
--
0 1
C   0.000000   0.000000   2.900503
C   0.000000   0.000000   1.693240
H   0.000000   0.000000   0.627352
H   0.000000   0.000000   3.963929
units angstrom
""")


In [9]:
psi4.set_options({'scf_type': 'df',
                  'freeze_core': True})

psi4.energy('sapt0/jun-cc-pvdz', molecule=dimer)

-0.0022355824076426816

In [10]:
#! Example potential energy surface scan and CP-correction for Ne2

ne2_geometry = """
Ne
--
Ne 1 {0}
"""

Rvals = [2.5, 3.0, 4.0]

psi4.set_options({'freeze_core': True})

# Initialize a blank dictionary of counterpoise corrected energies
# (Need this for the syntax below to work)

ecp = {}

for R in Rvals:
    ne2 = psi4.geometry(ne2_geometry.format(R))
    ecp[R] = psi4.energy('ccsd(t)/aug-cc-pvdz', bsse_type='cp', molecule=ne2)

# Prints to screen
print("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
print("          R [Ang]                 E_int [kcal/mol]       ")
print("---------------------------------------------------------")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    print("            {:3.1f}                        {:1.6f}".format(R, e))

# Prints to output.dat
psi4.core.print_out("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
psi4.core.print_out("          R [Ang]                 E_int [kcal/mol]       \n")
psi4.core.print_out("---------------------------------------------------------\n")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    psi4.core.print_out("            {:3.1f}                        {:1.6f}\n".format(R, e))

CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies


          R [Ang]                 E_int [kcal/mol]       
---------------------------------------------------------
            2.5                        0.758605
            3.0                        0.015968
            4.0                        -0.016215


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# Distances (R) and corresponding interaction energies (E) in kcal/mol
Rvals = np.array([3.5, 4.0, 5.0, 6.0])
Energies = np.array([ecp[R] * psi4.constants.hartree2kcalmol for R in Rvals])
# Define the Lennard-Jones potential function
def lj(r, sigma, eps):
    return 4 * eps * ((sigma / r)**12 - (sigma / r)**6)
# Fit the interaction energy data to the Lennard-Jones potential
params, covariance = curve_fit(lj, Rvals, Energies, p0=[3.8, -0.1])  # Initial guesses for sigma and eps
sigma_fit, eps_fit = params
# Generate fitted energy values over a range of distances for smooth plotting
R_fit = np.linspace(3.5, 6.5, 100)
E_fit = lj(R_fit, sigma_fit, eps_fit)
# Plot the computed energies and the fitted Lennard-Jones curve
plt.figure(figsize=(8, 6))
plt.plot(Rvals, Energies, 'o', label='Computed Energies', color='blue')  # Computed energies as dots
plt.plot(R_fit, E_fit, '--', label='LJ Fit', color='red')  # LJ fit as dashed line
# Add labels and legend
plt.xlabel('Distance (Å)', fontsize=12)
plt.ylabel('Interaction Energy (kcal/mol)', fontsize=12)
plt.title('Argon Dimer Interaction Energy vs. Distance', fontsize=14)
plt.legend()
plt.grid(True)
# Save the figure in the Figures directory
plt.savefig('Figures/argon_dimer_energy_vs_distance.png', dpi=300)
# Display the plot
plt.show()




KeyError: 4.0